In [1]:
import demjson
import json
import csv

In [2]:
country = 'mex'

In [3]:
en_file = './en-{}/translations.js'.format(country)
es_file = './es-{}/translations.js'.format(country)

In [4]:
def traverse(d, sep='.', _prefix=''):
    assert isinstance(d, dict)
    for k, v in d.items():
        if isinstance(v, dict):
            yield from traverse(v, sep, "{0}{1}{2}".format(_prefix, k, sep))
        else:
            yield ("{0}{1}".format(_prefix, k), v)

def flatten(d):
    return dict(traverse(d))

In [5]:
en_dict = demjson.decode_file(en_file)
es_dict = demjson.decode_file(es_file)

In [6]:
en_dict = flatten(en_dict)
len(en_dict)

251

In [7]:
es_dict = flatten(es_dict)
len(es_dict)

251

In [8]:
lang_csv = []
copy_keys = list(en_dict.keys()) + list(es_dict.keys())
for key in en_dict.keys():
    lang_csv.append({ "copy_key":key, "english": en_dict.get(key, ""), "spanish": es_dict.get(key, "")})

In [9]:
def write_to_csv(lang_csv, fieldnames):
    with open('translation.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames)
        writer.writeheader()
        for row in lang_csv:
            writer.writerow(row)
        

In [10]:
fieldnames = ['copy_key', 'english', 'spanish']

In [15]:
def read_from_csv(import_csv):
    translations= []
    spanish = {}
    english = {}
    
    with open(import_csv, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            translations.append(row)
    
    for row in translations:
        spanish[row['copy_key']] = row['spanish']
        english[row['copy_key']] = row['english']

    with open('new-es-{}.js'.format(country), 'w') as file:
        file.write(json.dumps(spanish, indent=2, sort_keys=True))
    with open('new-en-{}.js'.format(country), 'w') as file:
        file.write(json.dumps(english, indent=2, sort_keys=True))
        
    

In [12]:
#write_to_csv(lang_csv, fieldnames)

In [16]:
read_from_csv('translation.csv')